In [32]:
import pyodbc as odbc
import datetime

conn = odbc.connect("DSN=Local;Database=MetaData;APP=ETL")
curs = conn.cursor()

# Get the Procs
rows = curs.execute('SELECT ID, Name, ProcedureName FROM dbo.ETL ORDER BY ExecuteOrder').fetchall()

curs_audit = conn.cursor()
curs_proc = conn.cursor()
today = datetime.date.today()

# Loop through the procs
for row in rows:
    
    # Add Audit Record to Start Process
    curs_audit.execute('INSERT INTO dbo.ETLAudit(ETLID, RunDate, StartTime) VALUES(?, ?, GETDATE())', row[0], today)
    ident = curs_audit.execute('SELECT @@IDENTITY').fetchone()[0]
    curs_audit.commit()
    
    # Run the ETL Procedure
    curs_proc.execute("EXEC ?", row[2])
    curs_proc.commit()
    
    # Update Audit Record to End Process
    curs_audit.execute('UPDATE dbo.ETLAudit SET EndTime = GETDATE() WHERE ID = ?', ident)
    curs_audit.commit()
    
conn.close()

4
5


## Table for Procedures
Put your ETL procs in here

CREATE TABLE [dbo].[ETL](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[Name] [varchar](100) NOT NULL,
	[ProcedureName] [varchar](100) NOT NULL,
    ExecuteOrder INT NOT NULL
) ON [PRIMARY]

## Table for Audit

CREATE TABLE [dbo].[ETLAudit](
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[ETLID] [int] NOT NULL,
	[RunDate] [date] NOT NULL,
	[StartTime] [datetime] NOT NULL,
	[EndTime] [datetime] NULL
) ON [PRIMARY]